In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import re

In [3]:
#Read in the original csv file
KansasCity311 = pd.read_csv('../data/Kansas_City_MO_311.csv', low_memory=False)

In [4]:
KansasCity311.info() #960502 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960502 entries, 0 to 960501
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   CASE ID                 960502 non-null  int64  
 1   SOURCE                  960488 non-null  object 
 2   DEPARTMENT              960502 non-null  object 
 3   WORK GROUP              960502 non-null  object 
 4   REQUEST TYPE            960502 non-null  object 
 5   CATEGORY                960502 non-null  object 
 6   TYPE                    960502 non-null  object 
 7   DETAIL                  960502 non-null  object 
 8   CREATION DATE           960502 non-null  object 
 9   CREATION MONTH          960502 non-null  int64  
 10  CREATION YEAR           960502 non-null  int64  
 11  STATUS                  960502 non-null  object 
 12  EXCEEDED EST TIMEFRAME  960487 non-null  object 
 13  CLOSED DATE             929529 non-null  object 
 14  CLOSED MONTH        

In [5]:
#convert times to datetimes
KansasCity311['CREATION DATE'] = pd.to_datetime(KansasCity311['CREATION DATE'])

In [21]:
#what is the date range covered in this current dataframe?
#2006 - Part of June 2016
KansasCity311 = KansasCity311.sort_values(['CREATION DATE'],ascending=False)
KansasCity311.head()

,CASE ID,SOURCE,DEPARTMENT,WORK GROUP,REQUEST TYPE,CATEGORY,TYPE,DETAIL,CREATION DATE,CREATION MONTH,...,DAYS TO CLOSE,STREET ADDRESS,ADDRESS WITH GEOCODE,ZIP CODE,NEIGHBORHOOD,COUNCIL DISTRICT,PARCEL ID NO,XCOORDINATE,YCOORDINATE,CASE URL
960501,2016074055,PHONE,Public Works,Public Works-Street and Traffic-District 2,Streets / Roadways / Alleys-Street Clean Up-Di...,Streets / Roadways / Alleys,Street Clean Up,District 2,2016-06-28,6,...,NaN,5516 E 35TH ST,"5516 E 35TH ST\n64128\n(39.061527, -94.519244)",64128.0,Dunbar,3.0,23495,2.783258e+06,1.054167e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
960349,2016073841,PHONE,NHS,NHS-Solid Waste-,Trash / Recycling-Recycling-Missed by Contract...,Trash / Recycling,Recycling,Missed by Contractor South,2016-06-28,6,...,NaN,8718 ARLINGTON AVE,"8718 ARLINGTON AVE\n64138\n(38.964643, -94.470...",64138.0,White Oak,5.0,66986,2.797125e+06,1.018762e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
960340,2016073829,EMAIL,NHS,NHS-Solid Waste-Administration,Trash / Recycling-Services-Feedback,Trash / Recycling,Services,Feedback,2016-06-28,6,...,0.0,3939 GENESSEE ST,"3939 GENESSEE ST\n64111\n(39.055548, -94.605069)",64111.0,Volker,4.0,130554,2.758959e+06,1.051896e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
960300,2016073760,PHONE,NHS,NHS-Neighborhood Preservation-,Property Violations,Trash / Recycling,Nuisance,Trash on Private Property,2016-06-28,6,...,NaN,1917 LAWN AVE,"1917 LAWN AVE\n64127\n(39.08787, -94.528091)",64127.0,East Community Team North,3.0,18184,2.780823e+06,1.063662e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
960350,2016073843,PHONE,NHS,NHS-Animal Health and Safety-,Animals / Pets-Stray-At Large - Non-Aggressive,Animals / Pets,Stray,At Large - Non-Aggressive,2016-06-28,6,...,NaN,1345 E 75TH ST,"1345 E 75TH ST\n64131\n(38.991386, -94.571287)",64131.0,East Meyer 6,5.0,115998,2.768446e+06,1.028438e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...


## Cicada Time and Data

In [43]:
#Cicadas came to Missouri in 2015, via two broods at once: Brood XXIII and Brood IV.  
#Kansas City is in Jackson County which has confirmed sightings.


KansasCity311trim = KansasCity311[KansasCity311['CREATION DATE'] > dt.datetime(2013,5,1)]
KansasCity311trim = KansasCity311trim[KansasCity311trim['CREATION DATE'] < dt.datetime(2017,7,1)] 

## Isolating data to months of May and June for each year.  Cicada time.



In [8]:
#2013
KC2013 = KansasCity311trim[KansasCity311trim['CREATION DATE'] > dt.datetime(2013,5,1)]
KC2013 = KC2013[KC2013['CREATION DATE'] < dt.datetime(2013,7,1)]
KC2013 = KC2013.sort_values(['CREATION DATE'],ascending=False)
KC2013

,CASE ID,SOURCE,DEPARTMENT,WORK GROUP,REQUEST TYPE,CATEGORY,TYPE,DETAIL,CREATION DATE,CREATION MONTH,...,DAYS TO CLOSE,STREET ADDRESS,ADDRESS WITH GEOCODE,ZIP CODE,NEIGHBORHOOD,COUNCIL DISTRICT,PARCEL ID NO,XCOORDINATE,YCOORDINATE,CASE URL
684115,2013091062,WEB,NHS,NHS-Neighborhood Preservation-,Property Violations,Property Violations,Private Property,All,2013-06-30,6,...,164.0,9704 N CHARLOTTE ST,"9704 N CHARLOTTE ST\n64155\n(39.269597, -94.57...",64155.0,New Mark,2.0,103580,2.768691e+06,1.129827e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684138,2013091096,WEB,Public Works,Public Works-Solid Waste-,Dead Animal Pick-up,Trash,Animal,Dead,2013-06-30,6,...,2.0,100 W 90TH TER,"100 W 90TH TER\n64114\n(38.964107, -94.592324)",64114.0,Boone Hills,6.0,150094,2.762502e+06,1.018692e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684123,2013091077,PHONE,Parks and Rec,Parks and Rec-Landscape Services-Forestry,Tree Removal of a City Tree,Trees,Removal,*Select one,2013-06-30,6,...,1.0,4800 E 63RD ST,4800 E 63RD ST\n64130\n,64130.0,Swope Parkway-Elmwood,5.0,492,2.780203e+06,1.036936e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684128,2013091083,WEB,NHS,NHS-Neighborhood Preservation-,Property Violations,Property Violations,Private Property,All,2013-06-30,6,...,52.0,411 W 86TH ST,"411 W 86TH ST\n64114\n(38.972676, -94.596424)",64114.0,Western Hills,6.0,148749,2.761284e+06,1.021626e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
684135,2013091093,WEB,NHS,NHS-Animal Health and Safety-,Barking Dog,Animal,Barking Dog,All,2013-06-30,6,...,1.0,4621 NE 42ND ST,"4621 NE 42ND ST\n64117\n(39.170141, -94.524249)",64117.0,Winnwood,1.0,76201,2.781799e+06,1.093546e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665578,2013057954,PHONE,NHS,NHS-Animal Health and Safety-,Animal At Large,Animal,Owned or Stray at Large,All,2013-05-02,5,...,1.0,5600 E GREGORY BLVD,"5600 E GREGORY BLVD\n64132\n(38.996176, -94.52...",64132.0,NaN,5.0,44,2.782169e+06,1.030142e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
665577,2013057953,PHONE,NHS,NHS-Neighborhood Preservation-,Property Violations,Property Violations,Private Property,All,2013-05-02,5,...,75.0,934 E 76TH TER,"934 E 76TH TER\n64131\n(38.988892, -94.576529)",64131.0,Tower Homes,5.0,117666,2.766967e+06,1.027713e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
665576,2013057951,PHONE,NHS,NHS-Neighborhood Preservation-,Property Violations,Property Violations,Private Property,All,2013-05-02,5,...,5.0,4000 OLIVE ST,"4000 OLIVE ST\n64130\n(39.053572, -94.555989)",64130.0,Ivanhoe Southeast,3.0,136441,2.772705e+06,1.051183e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...
665575,2013057950,SYS,NHS,NHS-Neighborhood Preservation-,Property Violations,Property Violations,Private Property,All,2013-05-02,5,...,NaN,510 W 77TH ST,"510 W 77TH ST\n64114\n(38.988779, -94.596004)",64114.0,Ward Parkway,6.0,119798,2.761432e+06,1.027695e+06,http://webfusion.kcmo.org/coldfusionapps/Actio...


In [9]:
#2014
KC2014 = KansasCity311trim[KansasCity311trim['CREATION DATE'] > dt.datetime(2014,5,1)]
KC2014 = KC2014[KC2014['CREATION DATE'] < dt.datetime(2014,7,1)]

In [10]:
#2015 Cicada year
KC2015 = KansasCity311trim[KansasCity311trim['CREATION DATE'] > dt.datetime(2015,5,1)]
KC2015 = KC2015[KC2015['CREATION DATE'] < dt.datetime(2015,7,1)]

In [11]:
#2016
KC2016 = KansasCity311trim[KansasCity311trim['CREATION DATE'] > dt.datetime(2016,5,1)]
KC2016 = KC2016[KC2016['CREATION DATE'] < dt.datetime(2016,7,1)]

In [ ]:
KC2016.head()

## Bring it into a single dataframe

In [12]:
years = [KC2013, KC2014, KC2015, KC2016]
KC311_final = pd.concat(years)

In [13]:
KC311_final = KC311_final.sort_values(['CREATION DATE'],ascending=True)

In [14]:
KC311_final.info #75111 rows

<bound method DataFrame.info of            CASE ID SOURCE      DEPARTMENT  \
665376  2013057496  PHONE    Public Works   
665415  2013057567  PHONE    Public Works   
665431  2013057594  PHONE             NHS   
665413  2013057565  PHONE             NHS   
665422  2013057577  EMAIL  Water Services   
...            ...    ...             ...   
960471  2016074006  PHONE  Water Services   
960469  2016073999  PHONE   Parks and Rec   
960442  2016073956  PHONE   Parks and Rec   
960444  2016073960  PHONE   Parks and Rec   
960501  2016074055  PHONE    Public Works   

                                        WORK GROUP  \
665376     Public Works-Solid Waste-Administration   
665415  Public Works-Street and Traffic-District 3   
665431               NHS-Animal Health and Safety-   
665413               NHS-Animal Health and Safety-   
665422    Water Services-Line Maintenance-Pipeline   
...                                            ...   
960471  Water Services-Line Maintenance-Wastewate

## Get the relevant request types

In [15]:
KC311_final['REQUEST TYPE'].unique()

array(['Solid Waste Customer Service',
       'Pothole (South of 47th Street and West Of Blue PKWY)',
       'Animal Investigation', 'Animal At Large',
       'Pipeline Repair or Restoration Concerns', 'Property Violations',
       'Abandoned Vehicle', 'Traffic Signal Malfunction',
       'Sewer Water in Basement - Referral',
       'Animal Control Administration',
       'Catch Basins and Storm Water Concerns',
       'Street Maintenance General (North of the River)',
       'Street Maintenance General (South of 47th Street and West of Blue PKWY)',
       'Illegal Dumping on Property', 'Trash Missed by Contractor North',
       'Service or Curb Box Repair Referral', 'Meter Leak or Problem',
       'Stray Animal Confined', 'Dead Animal Pick-up',
       'Dangerous Building', 'Trash Recycling Missed by City',
       'Water Main Projects', 'Trash - Missed by City',
       'Trash Recycle Missed by Contractor North',
       'Water Leak or Pressure Problem',
       'Traffic Engineering and T

In [51]:
# What types of Phone Calls indicate bad behavior and/or an annoyed populace?  Excluding Emergency-related things and burnt-out streetlights
relevant_requests = [
    'Rural ROW Mowing', 'SLD0 Streetlight Out - Multiple Outage of Five or More','Property Violations', 'Building Code and Permit Violations',
    'Dead Animal Pick-up', 'Street Maintenance General (North of the River)','Public Health-Noise-Other',
     'Tree Removal of a City Tree', 'Pothole (North of River)', 'Ditch Cleaning (North from 27th Street )', 'Sidewalks / Curbs / Ditch-Damage / Dis-Repair-City',
     'Barking Dog', 'Parking Ticket Complaint', 'Animals / Pets-Bite-Owned', 'Animals / Pets-Cruelty or Neglect-Shelter / Food / Water',
      'Noise Complaints', 'Mowing on the Right of Way', 'Alley Cleaning (South of 47th Street and West of BLue PKWY)', 'Signs-Traffic Sign-Missing � Non-Emergency'
     'Abandoned Vehicle', 'Sign Violation Zoning', 'Graffiti on City Property', 'Mud and silt', 'Mowing / Weeds-Parks-North',
       'SLA0 Streetlight Pole Down or Leaning', 'Animal Bite', 'Public Restroom Sanitation','Trash-Staff Compliments for Solid Waste', 'Trash / Recycling-Nuisance-Early Set Out',
      'Animal Investigation','Sign Missing or Damaged - Non-emergency','Trash Missed by Contractor North',
    'Water Leak or Pressure Problem','Wildlife', 'SLG2 Streetlight Burning During Day','Animals / Pets-Questionable Activity-Cock fighting',
       'Animals / Pets-Stray-At Large - Highway', 'Public Health-Noise-Music', 'Public Health-Noise-Vehicles',
       'Trash / Recycling-Dumping-Park - South','Lights / Signals-Traffic Signal-Signal timing','Water-Quality-Taste','Public Health-Smoking / Tobacco-Restaurant',
       'Signs-Street Name Signs-Graffiti','Prop/Build/Construct - Land Use / Zoning - Unapproved Land Use',
       'Animals / Pets-Stray-At Large - Non-Aggressive', 'Trash / Recycling-Trash Collection-Missed by Contractor North','Trash / Recycling-Leaf / Brush-Missed Pick Up',
       'Streets / Roadways / Alleys-Broken Asphalt-District 2', 'Vehicles / Parking-Abandoned On Street-Car', 'Animals / Pets-Cruelty or Neglect-Abuse / Injury / Death',
       'Animals / Pets-Cruelty or Neglect-Inclement Weather', 'Trash / Recycling-Recycling-Missed by Contractor North','Vehicles / Parking-Abandoned On Street-Boat',
       'Trash / Recycling-Dumping-Camera Request', 'Lights / Signals-Street Light-Burning during the day','Public Health-Hotel / Motel-Unsanitary Conditions',
       'Animals / Pets-Services-Barking Dog Letter', 'Animals / Pets-Dead Animal -Curb', 'Mowing / Weeds-Alley-District 3','Vehicles / Parking-Abandoned On Street-Trailer',
       'Trash / Recycling-Bulky Pick Up -Missed Pick Up','Animals / Pets-Cruelty or Neglect-Abandonment','Trash / Recycling-Recycling-Repeat Missed',
       'Trash / Recycling-Recycling-Missed by City', 'Animals / Pets-Services-Off Leash - Investigation', 'Parks & Recreation-Graffiti-North',
       'Trash / Recycling-Services-Feedback', 'Signs-Street Name Signs-Missing','Mowing / Weeds - Visibility - Intersection',
       'Lights / Signals-Street Light-Graffiti','Public Health-Smoking / Tobacco-Public Building','Sidewalks / Curbs / Ditch-Unapproved Objects-Merchandise',
       'Mowing / Weeds-Blvd / Pkwy-North','Vehicles / Parking-Illegally Parked-Street/Sidewalk','Public Health-Noise-Trash Truck',
       'Animals / Pets-Questionable Activity-Other', 'Signs-Traffic Sign-Graffiti', 'Public Health-Noise-All', 'Trash / Recycling-Trash Cart-Not Delivered',
       'Trash / Recycling-Leaf / Brush-Dumping in Street','Trees-Care-Vandalism',
       'Parks & Recreation-Graffiti-Central','Trash / Recycling-Dumpsters-Street / Alley / ROW',
       'Animals / Pets-Stray-At Large - Owned', 'Trash / Recycling-Recycling-Missed by Contractor South',
       'Animals / Pets-Wildlife-Inside', 'Trash / Recycling-Trash Collection-Repeat Missed', 'Sidewalks / Curbs / Ditch-Unapproved Objects-Other',
       'Trash / Recycling-Dumping-Park - North', 'Vehicles / Parking-Abandoned On Street-Other',
       'Sidewalks / Curbs / Ditch-Unapproved Objects-Basketball Hoop/Goal',
       'Trash / Recycling-Services-Service Issue / Problem','Water-Quality-Smell / Odor','Trash / Recycling-Dumping-Right of Way',
       'Animals / Pets-Stray-At Large - Aggressive',
      'Food Establishment Complaints', 'Smoking Complaints','Graffiti on Signs', 'Issues on City Maintained Property', 'Streets / Roadways / Alleys-Street Services-Service Issue / Problem',
                     'Graffiti on Private Property', 'Outdoor Air Quality', 'Graffiti on Signs',
                     'Animal Control Administration', 'Tree Pruning for Block', 'Homeless Camp', 'Zoning Issue-Business in a residence',
       'Public Safety-Police-Homeless camp', 'Trash / Recycling-Dumping-Right of Way (ROW)',
                     'Pothole (River south to 47th Street and East of Blue PKWY)', 'Trees-Trimming-Tree Limbs',
                     'SLG0 Streetlight Out - Single Light Out', 'Trash / Recycling-Dumping-Private Property',
                     'SLP0 Streetlight Miscellaneous', 'Prop/Build/Construct-Construction Issue/Concern-Mud / Silt on street',
                     'Tree Trimming for City Tree', 'Traffic Signal Pole Damage, Rust, Vandalism',
                     'Sewer Cleaning or Repair and Waste Water Concern', 'Prop/Build/Construct-Land Use / Zoning Issue-Number of Dwelling Units',
                     'Trash Recycle Missed by Contractor South', 'No Water Bill Received',
       'Trash Cart Damaged','Bulky Pick-up Missed', 'Mowing / Weeds-Public Property-Land Bank', 'Trash / Recycling-Nuisance-Land Bank',
                     'Illegal Dumping on Property', 'Mowing / Weeds-Public Property-City Facility',
                     'Illegal Dumping on Right of Way', 'Mowing / Weeds-Right of Way (ROW)-Region - North',
                     'Trash Missed by Contractor South', 'Trash / Recycling-Dumping-Alley',
                     'Trash - Missed by City', 'Prop/Build/Construct-Construction Issue/Concern-Work without permit',
                     'Alley Cleaning (South of River to 47th Street and East of Blue PKWY)',
                     'Trash Recycle Missed by Contractor North',
                     'Tree Limb or Brush Removal for City Tree', 'Trees-Land Bank-Dead',
                     'Injury or Cruelty Involving an Animal','Animal At Large','Catch Basins and Storm Water Concerns',
       'Bulky Address Validation', 'Trash Recycling Missed by City',
       'Sewer Deodorizing - Referral', 'Violations and Encroachments on the Right of Way', 
                     'Graffiti on Streetlight Pole', 'Dangerous Building', 'Tree Trimming at Intersection', 'Stray Animal Confined',
       'Trash - Staff Complaints for Solid Waste', 'SLG1 Streetlight Blinking Off and On', 'Sidewalk or Curb in Disrepair', 'Bedbugs Problem',
                     'Vehicle Illegally Parked', 'Street Sign Damage', 'Tree Stump Removal for City Tree',
       'Ditch Cleaning (South of 27th Street)', 'Pothole (South of 47th Street and West Of Blue PKWY)', 'Tree Disease, Insects or Vandalism',
    'Vehicles / Parking-Commercial Vehicle-On private property',
       'Government-Public Works-Service Issue / Problem',
       'Trees-Land Bank-Tree Debris', 'Trash / Recycling-Trash Collection-Missed by City', 'Sidewalks / Curbs / Ditch-Unapproved Objects-Landscaping',
       'Mowing / Weeds-Parks-South', 'Trees-Trimming-Seasonal','Mowing / Weeds-Parks-Central','Mowing / Weeds-Blvd / Pkwy-Central',
       'Trees-Services-Service Issue / Problem',
       'Animals / Pets-Stray-Injured','Mowing / Weeds-Right of Way (ROW)-Region - South','Mowing / Weeds-Alley-District 2','Mowing / Weeds-Right of Way (ROW)-Region - Central','Trash / Recycling-Dumping-Right of Way'   
]

In [17]:
#trim dataframe to match the list of relevant request types

KC311_final = KC311_final[(KC311_final['REQUEST TYPE'].isin(relevant_requests))]

In [18]:
KC311_final.info() #50733 rows! 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50733 entries, 665415 to 960442
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   CASE ID                 50733 non-null  int64         
 1   SOURCE                  50730 non-null  object        
 2   DEPARTMENT              50733 non-null  object        
 3   WORK GROUP              50733 non-null  object        
 4   REQUEST TYPE            50733 non-null  object        
 5   CATEGORY                50733 non-null  object        
 6   TYPE                    50733 non-null  object        
 7   DETAIL                  50733 non-null  object        
 8   CREATION DATE           50733 non-null  datetime64[ns]
 9   CREATION MONTH          50733 non-null  int64         
 10  CREATION YEAR           50733 non-null  int64         
 11  STATUS                  50733 non-null  object        
 12  EXCEEDED EST TIMEFRAME  50733 non-null  

## Export .csv


In [54]:
KC311_final.to_csv('KC311_final.csv', index = False, header=True)

---------------------------------------

## found a new dataset which might include 2017

In [31]:
#New data source https://data.kcmo.org/311/311-Call-Center-Service-Requests-2007-March-2021/7at3-sxhp
#Read in the original csv file
SecondKC311 = pd.read_csv('../data/KC_second_dataset.csv', low_memory=False)

In [ ]:
SecondKC311.info() #1563077 rows!!

In [33]:
#convert to datetime
SecondKC311['CREATION DATE'] = pd.to_datetime(SecondKC311['CREATION DATE'])

In [34]:
SecondKC311 = SecondKC311.sort_values(['CREATION DATE'],ascending=False)

In [ ]:
SecondKC311.head() #yes! dates from this set go to 2021.  Retrimming from here

In [38]:
SecondKC311trim = SecondKC311[SecondKC311['CREATION DATE'] > dt.datetime(2013,5,1)]
SecondKC311trim = SecondKC311trim[SecondKC311trim['CREATION DATE'] < dt.datetime(2017,7,1)] 

In [ ]:
SecondKC311trim.info() #403830 rows

## Getting specific cicada months.  reusing variables from fist data set

In [44]:
#2013
KC2013 = SecondKC311trim[SecondKC311trim['CREATION DATE'] > dt.datetime(2013,5,1)]
KC2013 = KC2013[KC2013['CREATION DATE'] < dt.datetime(2013,7,1)]
KC2013 = KC2013.sort_values(['CREATION DATE'],ascending=False)

In [45]:
#2014
KC2014 = SecondKC311trim[SecondKC311trim['CREATION DATE'] > dt.datetime(2014,5,1)]
KC2014 = KC2014[KC2014['CREATION DATE'] < dt.datetime(2014,7,1)]

In [46]:
#2015 Cicada year
KC2015 = SecondKC311trim[SecondKC311trim['CREATION DATE'] > dt.datetime(2015,5,1)]
KC2015 = KC2015[KC2015['CREATION DATE'] < dt.datetime(2015,7,1)]

In [47]:
#2016
KC2016 = SecondKC311trim[SecondKC311trim['CREATION DATE'] > dt.datetime(2016,5,1)]
KC2016 = KC2016[KC2016['CREATION DATE'] < dt.datetime(2016,7,1)]

In [48]:
#2017
KC2013 = SecondKC311trim[SecondKC311trim['CREATION DATE'] > dt.datetime(2013,5,1)]
KC2013 = KC2013[KC2013['CREATION DATE'] < dt.datetime(2013,7,1)]
KC2013 = KC2013.sort_values(['CREATION DATE'],ascending=False)

In [ ]:
#bring them together
years = [KC2013, KC2014, KC2015, KC2016]
KC311_final = pd.concat(years)
KC311_final = KC311_final.sort_values(['CREATION DATE'],ascending=True)
KC311_final.info() #76387 rows 

In [50]:
KC311_final['REQUEST TYPE'].unique()
#Going to assume same request types and use same list as before

array(['Stray Animal Confined', 'Property Violations', 'Animal At Large',
       'Animal Investigation', 'Solid Waste Customer Service',
       'Sewer Repair Referral', 'Noise Complaints',
       'Catch Basins and Storm Water Concerns',
       'Pothole (South of 47th Street and West Of Blue PKWY)',
       'Tree Damage to City Tree from Storm Event',
       'Water Leak or Pressure Problem',
       'Sewer Water in Basement - Referral',
       'Street Maintenance General (North of the River)',
       'Ditch Cleaning (North from 27th Street )',
       'Water Services Administration', 'Meter Leak or Problem',
       'Pipeline Repair or Restoration Concerns',
       'Sidewalk or Curb in Disrepair', 'Vacant Structure Open to Entry',
       'Traffic Engineering and Traffic Studies',
       'Injury or Cruelty Involving an Animal',
       'Traffic Signal Malfunction', 'Trash Missed by Contractor South',
       'Zoning Inspections and Violations',
       'SLG2 Streetlight Burning During Day',
   

In [52]:
KC311_final = KC311_final[(KC311_final['REQUEST TYPE'].isin(relevant_requests))]

In [53]:
KC311_final.info() #51444 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51444 entries, 838508 to 1128523
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   CASE ID                    51444 non-null  int64         
 1   SOURCE                     51441 non-null  object        
 2   DEPARTMENT                 51444 non-null  object        
 3   WORK GROUP                 51444 non-null  object        
 4   REQUEST TYPE               51444 non-null  object        
 5   CATEGORY                   51444 non-null  object        
 6   TYPE                       51444 non-null  object        
 7   DETAIL                     51444 non-null  object        
 8   CREATION DATE              51444 non-null  datetime64[ns]
 9   CREATION TIME              51444 non-null  object        
 10  CREATION MONTH             51444 non-null  int64         
 11  CREATION YEAR              51444 non-null  int64         
 1